# Домашняя работа 1


In [ ]:
# !pip install torch torchvision torchaudio torch-geometric ogb --quiet


In [ ]:
from ogb.nodeproppred import NodePropPredDataset
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community.quality import modularity

import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np



1. Загрузка данных

In [ ]:
dataset = NodePropPredDataset(name='ogbn-arxiv', root='data')
graph, labels = dataset[0]

print("Количество узлов:", graph["num_nodes"])
print("Количество рёбер:", graph["edge_index"].shape[1])


In [ ]:
edge_index = graph["edge_index"].T  # np.array (1.1M × 2)
G = nx.DiGraph()
G.add_edges_from(edge_index)
G = G.to_undirected()

Визуализируйте граф на семпле из 1000 узлов

In [ ]:
sampled_nodes = list(G.nodes)[:1000:]
G_sampled = G.subgraph(sampled_nodes)
nx.draw(G_sampled, node_size=15)
plt.show()

### Предварительный анализ

Произведем предварительный анализ графа

In [ ]:
# TODO: средняя степень
number_of_nodes = G.number_of_nodes()
avg_degree = sum(dict(G.degree()).values()) / number_of_nodes

# TODO: плотность графа
density = 2 * G.number_of_edges() / (number_of_nodes * (number_of_nodes - 1))

# TODO: диаметер графа
diameter = nx.diameter(G)

# TODO: радиус графа
radius = nx.radius(G)

print(f"Средняя степень: {avg_degree:.2f}")
print(f"Плотность: {density:.6f}")
print(f"Диаметер графа: {diameter:.4f}")
print(f"Радиус графа: {radius:.4f}")

### Анализ на центральность

Найдите узел с максимальной центральностью

In [ ]:
# TODO: Вычисление центральности по степени (degree centrality)
degree_centrality = nx.degree_centrality(G)
print("Центральность по степени:", degree_centrality)

# TODO: Вычисление центральности по близости (closeness centrality)
closeness_centrality = nx.closeness_centrality(G)
print("Центральность по близости:", closeness_centrality)

# TODO: Вычисление центральности по междуузловой значимости (betweenness centrality)
betweenness_centrality = nx.betweenness_centrality(G)
print("Центральность по междуузловой значимости:", betweenness_centrality)


Анализ кластеров

In [ ]:
# TODO: Вычислите средний кластерный коэффициент графа
global_cluster_coeff = nx.average_clustering(G)
print("Cредний кластерный коэффициент графа:", global_cluster_coeff)


In [ ]:
# TODO: Вычислите коэффициент транзитивности
transitivity_coeff = nx.transitivity(G)
print("Коэффициент транзитивности графа:", transitivity_coeff)


### Поиск сообществ

Реализуйте 2 метода поиска сообществ:
1. С помощью готовой реализации в библиотеке NetworkX
2. Реализуйте самостоятельно класс с алгоритмом Clauset–Newman–Moore

Сравните полученные результаты

Алгоритм Clauset–Newman–Moore (CNM)

---

**1. Начальное состояние**
- Каждая вершина — отдельное сообщество.  
- Вычисляем степени вершин и матрицу смежности.

---

**2. Вычисляем для всех пар сообществ** $(r, s)$, между которыми есть хотя бы одно ребро:

$$
\Delta Q_{rs} = 2 \, (e_{rs} - a_r a_s)
$$

где:  

- $e_{rs}$ — доля рёбер между сообществами *r* и *s*, делённая на $2m$;  
- $a_r = \sum_t e_{rt}$ — доля концов рёбер, инцидентных *r*.

---

**2.1. Доля концов рёбер, инцидентных вершинам из $r$**

Это часть всех концов рёбер в графе, которые принадлежат вершинам, входящим в сообщество $r$.

Эта величина обозначается как $a_r$ и вычисляется по формуле:

$$
a_r = \frac{1}{2m} \sum_{i \in r} k_i
$$

где:  

- $k_i$ — степень вершины $i$ (сколько у неё рёбер);  
- $m$ — общее число рёбер в графе;  
- $2m$ — общее количество **концов рёбер** (так как каждое ребро имеет два конца).

---

**3. Выбираем пару $(r, s)$** с максимальным $\Delta Q$ и объединяем их.

---

**4. Обновляем:**
- значения $e_{rt}, a_r$;  
- новую модульность:  
  $$
  Q \leftarrow Q + \Delta Q
  $$

---

**5. Повторяем**, пока $\Delta Q > 0$.

---

**6. Выбираем состояние с максимальным $Q$** как оптимальное разбиение.


Описание алгоритма - https://arxiv.org/pdf/cond-mat/0408187

In [ ]:
class CNM:
    """
    Алгоритм Clauset–Newman–Moore для обнаружения сообществ.
    TODO: Заполните пропущенные участки кода.
    """

    def __init__(self, edges):
        self.graph = defaultdict(dict)
        for u, v, w in edges:
            if u == v: continue
            self.graph[u][v] = self.graph[u].get(v, 0) + w
            self.graph[v][u] = self.graph[v].get(u, 0) + w
        self.nodes = list(self.graph.keys())
        self.m = sum(sum(self.graph[u].values()) for u in self.nodes) / 2
        self.deg = {u: sum(self.graph[u].values()) for u in self.nodes}

    def fit(self, verbose=False):
        communities = {u: {u} for u in self.nodes}
        comm_id = {u: u for u in self.nodes}
        m2 = 2 * self.m
        a = {u: self.deg[u] / m2 for u in self.nodes}

        # TODO 0: Доля рёбер между сообществами
        e = defaultdict(lambda: defaultdict(float))
        for u in self.nodes:
            for v, w in self.graph[u].items():
                if u != v:
                    e[u][v] += w
        for r in e:
            for s in e[r]:
                e[r][s] /= m2

        # Начальная модульность
        Q = -sum(a[c] ** 2 for c in a)

        # Основной цикл
        while True:
            best_dQ = 0
            best_pair = None

            # TODO 1: Найти пару (r, s) с максимальным ΔQ
            for r in e:
                for s in e[r]:
                    if r != s:
                        delta_Q = 2 * (e[r][s] - a[r] * a[s])
                        if delta_Q > best_dQ:
                            best_dQ = delta_Q
                            best_pair = (r, s)

            # TODO 2: Условие выхода
            if best_dQ <= 0:
                break

            r, s = best_pair
            if verbose:
                print(f"Объединяем {r}, {s} с ΔQ={best_dQ:.6f}")

            # TODO 3: Объединение сообществ
            communities[r].update(communities[s])
            for node in communities[s]:
                comm_id[node] = r
            del communities[s]

            # TODO 4: Обновление a, e, Q
            a[r] += a[s]
            del a[s]
            for t in list(e[s].keys()):
                if t != r:
                    e[r][t] += e[s][t]
                    e[t][r] = e[r][t]
                del e[t][s]
            del e[s]
            Q += best_dQ

        final_comms = [c for c in communities.values() if c]
        return final_comms, Q


In [ ]:
model = CNM(edges)
comms, Q_my = model.fit(verbose=True)

# NetworkX эталон
nx_comms = list(greedy_modularity_communities(G))
Q_nx = modularity(G, nx_comms)

print("\n=== Сравнение ===")
print("CNM (наш)  Q =", round(Q_my, 5))
print("NetworkX    Q =", round(Q_nx, 5))
print("Количество сообществ:", len(comms))


Визуализация результатов

In [ ]:
plt.figure(figsize=(20, 10))
sub_nodes = list(G.nodes)[:1000]
H = G.subgraph(sub_nodes)

# создаём отображение: node -> community_id
node2comm = {}
for i, comm in enumerate(nx_comms):
    for n in comm:
        node2comm[n] = i

colors = [node2comm.get(n, 0) for n in H.nodes()]
nx.draw(H, node_color=colors, node_size=40, edge_color="gray", with_labels=False, cmap="tab10")
plt.title("Гграф окрашен по сообществам CNM")
plt.show()